In [8]:
#GRUPO:
#Giovanna Arana Paganotti - 11315033
#Glaucia Pamponet Sobrinho - 11271000
#Thais Bonjorni Nobre de Jesus - 10719302


# **Imports e Métodos**

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from scipy.special import expit
from mlxtend.evaluate import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix
import plotly.figure_factory as ff
import seaborn as sn

In [10]:
def probsLogicos(problema):
    gdd.download_file_from_google_drive(file_id=google_id, dest_path = './problem'+problema+'.csv', showsize = True) #download no drive
    data = pd.read_csv("/content/drive/Shareddrives/Voodoo Chicken/5SEM/IA/MLP/CSVs/problem"+problema+".csv", header=None).to_numpy() #faz array para facilitar o uso
  # print('Dataset '+problema+":")
  # print(data, "\n\n")
  
  #recorte de entradas e saidas
    Xtrain = data[0:, 0:2].T
    Ytrain = data[0:, 2:3].T

  #cria o modelo
    modelo = MultPerc(2, 31, 1) #entrada, escondida, saida

    print("PESOS ESCONDIDA\n",modelo.Pesc.T)
    print("\nPESOS SAIDA\n",modelo.Psaida)
    print("\nBIAS ESCONDIDA\n",modelo.Besc.T)
    print("\nBIAS SAIDA\n",modelo.Bsaida.T)
    print("\n\n")

  # print(problema+" Antes do Treino:")
  # print(feedforward(modelo, Xtrain), "\n")

    back(modelo, Xtrain, Ytrain, 0.1, 0.001)	#modelo, entradasDataset, saidasDataset, taxa aprend, limiteErro

  # print("\n"+problema+" Depois do Treino:")
  # print(feedforward(modelo, Xtrain))

    return modelo

In [11]:
def fazBin(resultado):
  resultado2 = np.copy(resultado)
  rescop = resultado2.astype(int)
  for i in range(0, resultado2.shape[0]):
    r = 0
    for m in range(0, int(len(resultado2[i]))):
      if resultado2[i, m] > r:
        r = resultado2[i, m]
    ja = False
    for o in range(0, int(len(resultado2[i]))):
      if resultado2[i, o] == r and not ja:
        ja = True
        rescop[i, o] = int(1)
      else: rescop[i, o] = int(0)
    
  return rescop

# **Estrutura do Modelo**

In [12]:
class MultPerc():
    def __init__(self, entra, esc, said):
        self.entrada = entra
        self.escondida = esc
        self.saida = said
        self.Pesc, self.Besc = self.geraPesos(esc, entra)
        self.Psaida, self.Bsaida = self.geraPesos(said, esc)

    def geraPesos(self, lin, col):
        P = np.random.ranf((lin, col)) - 0.5
        B = np.ones((lin, 1))
        return P, B

    def guardResult(self, resesc, resFesc, resai, resFsai):
        self.resesc = resesc
        self.resFesc = resFesc
        self.resai = resai
        self.resFsai = resFsai
    
    def guardEpoca(self, epc, lista):
        self.epocas = epc
        self.listaErro = lista


# **Função de Ativação e Derivada**

In [13]:
def funcAtivacao (av):
    norberto = expit(av) #1/(1+exp(-x))
    return norberto
  
def derivada (av):
    return av - np.power(av, 2) #(funcAtivacao(av) * (1 - funcAtivacao(av)))

# **FeedForward**

In [14]:
def feedforward(modelo, linha):
  #passando pela escondida
  modelo.resesc = np.dot(modelo.Pesc, linha) + modelo.Besc
  modelo.resFesc = funcAtivacao(modelo.resesc) #RESULTADO DA ESCONDIDA 

  #passando pela saida
  modelo.resai = np.dot(modelo.Psaida, modelo.resFesc) + modelo.Bsaida
  res = funcAtivacao(modelo.resai) #RESULTADO DA SAIDA USANDO A ESCONDIDA
  modelo.resFsai = res

  return res

# **Retro-Propagação de Erro - BackPropagation**

In [15]:
def back(modelo, X, Y, learningRate, tresh):
  errorList = []
  tamData = int(len(X.T))
  erroCorr = 2*tresh
  epocas = 0

  #rodando epocas até o erro ser aceitavel
  while(erroCorr > tresh):
    #passa por cada linha do dataset enviado no parametro
    for i in range(0, tamData):
      caso = X[0:len(X), i:i+1]
      resposta = Y[0:len(Y), i:i+1]
      
      res = feedforward(modelo, caso)

      erroCorr += float(np.sum((np.sum((res - resposta)**2, axis = 0, keepdims = True)), axis = 1, keepdims = True))

      #TREINAMENTO DAS CAMADAS
      dsaida = -2*(res - resposta)*(derivada(res)) #VARIAÇÂO SAIDA
      dPsaida = np.dot(dsaida, modelo.resFesc.T)
      dBsaida = np.sum(dsaida, axis = 1, keepdims = True)

      desc = np.dot(modelo.Psaida.T, dsaida)*(derivada(modelo.resFesc)) #VARIAÇÃO ESCONDIDA
      dPesc = np.dot(desc, caso.T) 
      dBesc = np.sum(desc, axis = 1, keepdims = True)

      #ATUALIZAÇÃO NO MODELO
      modelo.Psaida += learningRate*dPsaida
      modelo.Bsaida += learningRate*dBsaida
      modelo.Pesc += learningRate*dPesc
      modelo.Besc += learningRate*dBesc

    erroCorr = erroCorr/tamData
    errorList.append(erroCorr) #lista com erros das epocas
    epocas += 1

    #print("ERRO:", erroCorr)

  modelo.guardEpoca(epocas, errorList)
  
  fig = plt.figure(figsize = (8,7))
  fig.patch.set_facecolor('white')
  plt.plot(range(0, modelo.epocas), modelo.listaErro, "r-")
  plt.grid(False)
  plt.title("Erro Por Epocas")
  plt.show()



# **Teste dos Problemas Lógicos**

### **XOR**

In [16]:
modelXOR = probsLogicos("XOR")

NameError: name 'gdd' is not defined

# **Dataset Caracteres Limpos**

### **Treinamento**

In [ ]:
gdd.download_file_from_google_drive(file_id=google_id, dest_path = './caracteres-limpo.csv', showsize = True)
dataLIMPO = pd.read_csv("/content/drive/Shareddrives/Voodoo Chicken/5SEM/IA/MLP/CSVs/caracteres-limpo.csv", header=None, low_memory=False)
dataLIMPO.head(dataLIMPO.shape[0])


#print(dataLIMPO, "\n\n")
dataLIMPO = dataLIMPO.to_numpy()

#SEPARAÇÃO DAS ENTRADAS E SAIDAS
Xtrain = dataLIMPO[0:, 0:63]
Ytrain = dataLIMPO[0:, 63:]

entreino, enteste, saitreino, saiteste = train_test_split(Xtrain, Ytrain, test_size=0.3)

particaoENT = pd.DataFrame(entreino)
particaoSAI = pd.DataFrame(saitreino)
particaoENT.to_csv('/content/drive/Shareddrives/Voodoo Chicken/5SEM/IA/MLP/CSVs/arquivoEnt.csv')
particaoSAI.to_csv('/content/drive/Shareddrives/Voodoo Chicken/5SEM/IA/MLP/CSVs/arquivoSai.csv')

# #cria o modelo
modelo = MultPerc(63, 31, 7)	#entrada, escondida, saida

# print(" Antes do Treino:\n")
# antesLIMPO = pd.DataFrame(forwardProp(modelo, entreino.T).T)
# print(antesLIMPO.head(antesLIMPO.shape[0]), "\n\n")

print("Resultados do Conjunto:\n")
saidaset = pd.DataFrame(saitreino)
print(saidaset.head(saidaset.shape[0]), "\n\n")

back(modelo, entreino.T, saitreino.T, 0.1, 0.001)	#modelo, entradasDataset, saidasDataset, taxa, limiteErro

print("\n Depois do Treino:\n")
limpoTREINO = pd.DataFrame(feedforward(modelo, entreino.T).T)
limpoTREINO.head(limpoTREINO.shape[0])

In [ ]:
jedo = fazLetra(fazBin(feedforward(modelo, entreino.T).T))
jeda = fazLetra(saitreino)
cm = confusion_matrix(jedo, jeda, binary=False)
uniao = np.union1d(jeda, jedo)
df_cm = pd.DataFrame(cm,  columns=np.unique(uniao), index = np.unique(uniao))
df_cm.index.name = 'Expectativa'
df_cm.columns.name = 'Realidade'
figu = plt.figure(figsize = (8,7))
figu.patch.set_facecolor('white')
plt.title("\nMatriz de Confusão\n")
sn.heatmap(df_cm, annot=True, linewidths=.5, cbar = False) 

### **Teste**

Resposta do Conjunto de Teste

In [ ]:

#TESTE LIMPO
jedo = fazLetra(saiteste) #transforma resposta certa em letra
print("Resultado certo:", jedo)
testeset = pd.DataFrame(saiteste)
print(testeset.head(saiteste.shape[0]), "\n\n")

print("Resultado Obtido:")
limpoTESTE = pd.DataFrame(feedforward(modelo, enteste.T).T)
print(limpoTESTE.head(limpoTESTE.shape[0]), "\n\n")

variavel = fazBin(limpoTESTE) #transforma resposta da rede em binario
jeda = fazLetra(variavel) #transforma binario em letra
print("Resultado Aproximado:", jeda)
print(pd.DataFrame(variavel).head(limpoTESTE.shape[0]))




In [ ]:
print(modelo.Pesc)
print(modelo.Besc)

### **Matriz de Confusão**

In [ ]:
#MATRIZ
cm = confusion_matrix(jedo, jeda, binary=False)

#UNIAO DOS LABELS PARA REPRESENTAR
uniao = np.union1d(jeda, jedo)

#REP GRAFICA
df_cm = pd.DataFrame(cm,  columns=np.unique(uniao), index = np.unique(uniao))
df_cm.index.name = 'Expectativa'
df_cm.columns.name = 'Realidade'
figu = plt.figure(figsize = (8,7))
figu.patch.set_facecolor('white')
plt.title("\nMatriz de Confusão Limpo\n")
sn.heatmap(df_cm, annot=True, linewidths=.5, cbar = False) 

# **Dataset Caracteres com Ruído**

### **Treinamento**

In [ ]:
gdd.download_file_from_google_drive(file_id=google_id, dest_path = './caracteres-ruido.csv', showsize = True)
dataRUIDO = pd.read_csv("/content/drive/Shareddrives/Voodoo Chicken/5SEM/IA/MLP/CSVs/caracteres-ruido.csv", header=None, low_memory=False)
dataRUIDO.head(dataRUIDO.shape[0])


#print(dataLIMPO, "\n\n")
dataRUIDO = dataRUIDO.to_numpy()

#SEPARA ENTRADA DA SAIDA
XtrainR = dataRUIDO[0:, 0:63]
YtrainR = dataRUIDO[0:, 63:]

#DIVISAO ALEATORIA DO DATASET EM PARTE DE TREINO E TESTE
entreinoR, entesteR, saitreinoR, saitesteR = train_test_split(XtrainR, YtrainR, test_size=0.3)

particaoENTr = pd.DataFrame(entreino)
particaoSAIr = pd.DataFrame(saitreino)
particaoENTr.to_csv('/content/drive/Shareddrives/Voodoo Chicken/5SEM/IA/MLP/CSVs/arquivoEntR.csv')
particaoSAIr.to_csv('/content/drive/Shareddrives/Voodoo Chicken/5SEM/IA/MLP/CSVs/arquivoSaiR.csv')

# #cria o modelo
modeloRuido = MultPerc(63, 31, 7)	#entrada, escondida, saida

# print(" Antes do Treino:\n")
# antesLIMPO = pd.DataFrame(forwardProp(modelo, entreino.T).T)
# print(antesLIMPO.head(antesLIMPO.shape[0]), "\n\n")

print("Resultados do Conjunto:\n")
saidasetR = pd.DataFrame(saitreinoR)
print(saidasetR.head(saidasetR.shape[0]), "\n\n")

back(modeloRuido, entreinoR.T, saitreinoR.T, 0.1, 0.001)	#modelo, entradasDataset, saidasDataset, taxa, limiteErro

print("\n Depois do Treino:\n")
ruidoTREINO = pd.DataFrame(feedforward(modeloRuido, entreinoR.T).T)
ruidoTREINO.head(ruidoTREINO.shape[0])

In [ ]:
print(modeloRuido.epocas)

Matriz de Confusao do Treinamento

In [ ]:
#UNIAO DAS RESPOSTAS PARA FAZER TODOS OS LABELS POSSIVEIS DA MATRIZ
jedo = fazLetra(fazBin(feedforward(modeloRuido, entreinoR.T).T))
jeda = fazLetra(saitreinoR)

#MONTA MATRIZ
cm = confusion_matrix(jedo, jeda, binary=False)
uniao = np.union1d(jeda, jedo)
df_cm = pd.DataFrame(cm,  columns=np.unique(uniao), index = np.unique(uniao))
df_cm.index.name = 'Expectativa'
df_cm.columns.name = 'Realidade'
figu = plt.figure(figsize = (8,7))
figu.patch.set_facecolor('white')
plt.title("\nMatriz de Confusão Treinamento Com Ruido\n")
sn.heatmap(df_cm, annot=True, linewidths=.5, cbar = False) 

### **Teste**

In [ ]:

#TESTE RUIDO
jedoR = fazLetra(saitesteR) #transforma resposta certa em letra
print("Resultado certo:", jedoR) 
testesetR = pd.DataFrame(saitesteR)
print(testesetR.head(saitesteR.shape[0]), "\n\n")

print("Resultado Obtido:")
ruidoTESTE = pd.DataFrame(feedforward(modeloRuido, entesteR.T).T)
print(ruidoTESTE.head(ruidoTESTE.shape[0]), "\n\n")

variavelR = fazBin(ruidoTESTE) #transforma resposta da rede em binario
jedaR = fazLetra(variavelR) #transforma binario em letra
print("Resultado Aproximado:", jedaR)
print(pd.DataFrame(variavelR).head(ruidoTESTE.shape[0]))


### **Matriz de Confusão**

In [ ]:
#MATRIZ
cm = confusion_matrix(jedoR, jedaR, binary=False)

#UNIAO DOS LABELS POSSIVEIS NA MATRIZ
uniao = np.union1d(jedaR, jedoR)

#REP GRAFICA
df_cm = pd.DataFrame(cm,  columns=np.unique(uniao), index = np.unique(uniao))
df_cm.index.name = 'Expectativa'
df_cm.columns.name = 'Realidade'
figu = plt.figure(figsize = (8,7))
figu.patch.set_facecolor('white')
plt.title("\nMatriz de Confusão com Ruido\n")
sn.heatmap(df_cm, annot=True, linewidths=.5, cbar = False) 